In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


データ読み込み

In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv("/content/drive/MyDrive/SIGNATE/Deloitte/train.csv")
test = pd.read_csv("/content/drive/MyDrive/SIGNATE/Deloitte/test.csv")

テストデータ結合

In [3]:
test_1 = pd.read_csv("/content/drive/MyDrive/SIGNATE/Deloitte/20230616_0116.csv",names=['id', 'attendance'])

In [4]:
test_2 = test.copy()
first_column = test_1.pop('attendance')
l = len(test_2.columns.values)
test_2.insert(l,"attendance",first_column)

In [5]:
#train =pd.concat([train, test_2])

In [6]:
display(train.tail())
display(test.head())

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,attendance
3361,18195,2016-11-03,13:33,第17節,第1日,川崎F,G大阪,等々力陸上競技場,晴,20.0,41,スカパー!/スカパー!プレミアムサービス/NHK BS1,25694
3362,18196,2016-11-03,13:33,第17節,第1日,甲府,鳥栖,山梨中銀スタジアム,晴,20.5,20,スカパー!/スカパー!プレミアムサービス,14676
3363,18197,2016-11-03,13:33,第17節,第1日,新潟,広島,デンカビッグスワンスタジアム,曇のち雨のち晴,18.2,33,スカパー!/スカパー!プレミアムサービス/NHK新潟/NHK広島,27081
3364,18198,2016-11-03,13:33,第17節,第1日,名古屋,湘南,パロマ瑞穂スタジアム,晴,17.0,41,スカパー!/スカパー!プレミアムサービス/NHK名古屋,18474
3365,18199,2016-11-03,13:33,第17節,第1日,福岡,柏,レベルファイブスタジアム,晴,18.3,50,スカパー!/スカパー!プレミアムサービス,13042


,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters
0,19075,2017-02-25,12:35,第1節,第1日,横浜FM,浦和,日産スタジアム,晴,10.2,45.0,DAZN/テレビ神奈川/テレ玉
1,19076,2017-02-25,14:04,第1節,第1日,仙台,札幌,ユアテックスタジアム仙台,曇,10.6,36.0,DAZN/NHK仙台/NHK札幌
2,19077,2017-02-25,14:04,第1節,第1日,鹿島,FC東京,県立カシマサッカースタジアム,晴,10.1,63.0,DAZN/NHK総合
3,19078,2017-02-25,14:04,第1節,第1日,清水,神戸,IAIスタジアム日本平,晴,15.0,40.0,DAZN/NHK BS1/テレビ静岡
4,19079,2017-02-25,14:04,第1節,第1日,広島,新潟,エディオンスタジアム広島,晴,11.2,45.0,DAZN/NHK広島/NHK新潟


In [7]:
sec_1 = list(set(train["round"].values.tolist()))
l =[]
for i in range(len(sec_1)):
    x = list(sec_1[i])
    x1 = sec_1[i].lstrip(x[0]).rstrip(x[len(x)-1])
    l.append(int(x1))
sec_1 = sorted(l)
print(sec_1)


[1, 2, 3]


日付・試合正規化

In [8]:
train["match_date"] = [int(x.replace("-","")) for x in train["match_date"].values]
train["kick_off_time"] = [int(x.replace(":","")) for x in train["kick_off_time"].values]
train["section"] = [int(x.replace("第","").replace("節","")) for x in train["section"].values]
train["round"] = [int(x.replace("第","").replace("日","")) for x in train["round"].values]
train.head()

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,attendance
0,9190,20060304,1604,1,1,G大阪,浦和,万博記念競技場,晴,8.3,40,NHK総合/J SPORTS(録),20916
1,9191,20060305,1300,1,2,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,28,山梨放送/テレビ静岡(録)/J SPORTS(録),14277
2,9192,20060305,1335,1,2,FC東京,大分,味の素スタジアム,晴,12.1,35,BS-i/MXテレビ(録)/J SPORTS(録),22531
3,9193,20060305,1404,1,2,磐田,福岡,静岡スタジアムエコパ,晴,11.6,42,J SPORTS,28564
4,9194,20060305,1404,1,2,名古屋,C大阪,名古屋市瑞穂陸上競技場,晴,13.1,32,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),17199


チーム名

In [9]:
home_team_key = sorted(set(train["home_team"].values.tolist()),key =train["home_team"].values.tolist().index)
sum(train["attendance"][train["home_team"] == home_team_key[0]].values)/len(train["attendance"][train["home_team"] == home_team_key[0]].values)
l =[]
a = []
for i in range(len(home_team_key)):
    ave = sum(train["attendance"][train["home_team"] == home_team_key[i]].values)/len(train["attendance"][train["home_team"] == home_team_key[i]].values)
    l.append(ave)
    a.append(ave)
a.sort()
home_t =[0]*len(l)
for i in range(len(l)):
    home_t[i] = a.index(l[i])

for i in range(len(l)):
    train["home_team"][train["home_team"] == home_team_key[i]] = int(home_t[i])



<ipython-input-9-7edecf7d9169>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["home_team"][train["home_team"] == home_team_key[i]] = int(home_t[i])


天気

In [10]:
for i in range(len(l)):
    train["away_team"][train["away_team"] == home_team_key[i]] = int(home_t[i])

<ipython-input-10-03f4bb34a39b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["away_team"][train["away_team"] == home_team_key[i]] = int(home_t[i])


In [11]:
weather_key = sorted(set(train["weather"].values.tolist()),key =train["weather"].values.tolist().index)
sum(train["attendance"][train["weather"] == weather_key[0]].values)/len(train["attendance"][train["weather"] == weather_key[0]].values)
l =[]
a = []
for i in range(len(weather_key)):
    ave = sum(train["attendance"][train["weather"] == weather_key[i]].values)/len(train["attendance"][train["weather"] == weather_key[i]].values)
    l.append(ave)
    a.append(ave)
a.sort()
weather_t =[0]*len(l)
for i in range(len(l)):
    weather_t[i] = a.index(l[i])

for i in range(len(l)):
    train["weather"][train["weather"] == weather_key[i]] = int(weather_t[i])



<ipython-input-11-c76da99eb20e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["weather"][train["weather"] == weather_key[i]] = int(weather_t[i])


会場情報追加

In [12]:
venue_info = pd.read_csv("/content/drive/MyDrive/SIGNATE/Deloitte/venue_information.csv")

In [13]:
venue_info.head()

,venue,capacity,address
0,万博記念競技場,21000,大阪府吹田市千里万博公園5-2
1,山梨県小瀬スポーツ公園陸上競技場,15859,山梨県甲府市小瀬町840
2,味の素スタジアム,48999,東京都調布市西町376-3
3,静岡スタジアムエコパ,51697,静岡県袋井市愛野2300−1
4,名古屋市瑞穂陸上競技場,20223,愛知県名古屋市瑞穂区山下通5-1


In [14]:
l =venue_info["address"].values

In [15]:
import re
DATA = """
北海道
青森県
岩手県
宮城県
秋田県
山形県
福島県
茨城県
栃木県
群馬県
埼玉県
千葉県
東京都
神奈川県
新潟県
富山県
石川県
福井県
山梨県
長野県
岐阜県
静岡県
愛知県
三重県
滋賀県
京都府
大阪府
兵庫県
奈良県
和歌山県
鳥取県
島根県
岡山県
広島県
山口県
徳島県
香川県
愛媛県
高知県
福岡県
佐賀県
長崎県
熊本県
大分県
宮崎県
鹿児島県
沖縄県
"""

prefecture = re.findall(r'(\w+)\n',DATA)
print(prefecture)

['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県']


In [16]:
for i in range(len(prefecture)):
    for j in range(len(l)):
        if prefecture[i] in venue_info["address"][j]:
            venue_info["address"][j]=prefecture[i]

<ipython-input-16-d9c0f67b5eec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venue_info["address"][j]=prefecture[i]


In [17]:
venue_info_key = sorted(set(venue_info["venue"].values.tolist()),key =venue_info["venue"].values.tolist().index)
train["capacity"]=0
train["address"] =0
test["capacity"]=0
test["address"] =0
for i in range(len(venue_info_key)):
    train["capacity"][train["venue"]==venue_info_key[i]] = venue_info["capacity"][i]
    test["capacity"][test["venue"]==venue_info_key[i]] = venue_info["capacity"][i]
    train["address"][train["venue"]==venue_info_key[i]] = venue_info["address"][i]
    test["address"][test["venue"]==venue_info_key[i]] = venue_info["address"][i]

train.head()

<ipython-input-17-6dd6d70b09d7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["capacity"][train["venue"]==venue_info_key[i]] = venue_info["capacity"][i]
<ipython-input-17-6dd6d70b09d7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["capacity"][test["venue"]==venue_info_key[i]] = venue_info["capacity"][i]
<ipython-input-17-6dd6d70b09d7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["address"][train["venue"]==ven

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,attendance,capacity,address
0,9190,20060304,1604,1,1,21,28,万博記念競技場,29,8.3,40,NHK総合/J SPORTS(録),20916,21000,大阪府
1,9191,20060305,1300,1,2,6,16,山梨県小瀬スポーツ公園陸上競技場,29,12.9,28,山梨放送/テレビ静岡(録)/J SPORTS(録),14277,15859,山梨県
2,9192,20060305,1335,1,2,26,24,味の素スタジアム,29,12.1,35,BS-i/MXテレビ(録)/J SPORTS(録),22531,48999,東京都
3,9193,20060305,1404,1,2,10,7,静岡スタジアムエコパ,29,11.6,42,J SPORTS,28564,51697,静岡県
4,9194,20060305,1404,1,2,19,18,名古屋市瑞穂陸上競技場,29,13.1,32,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),17199,20223,愛知県


In [18]:
l =[]
a = []
for i in range(len(prefecture)):
    if len(train["attendance"][train["address"] == prefecture[i]].values) !=0:
        ave = sum(train["attendance"][train["address"] == prefecture[i]].values)/len(train["attendance"][train["address"] == prefecture[i]].values)
    l.append(ave)
    a.append(ave)
a.sort()
venue_i_t =[0]*len(l)
for i in range(len(l)):
    venue_i_t[i] = a.index(l[i])

for i in range(len(l)):
    train["address"][train["address"] == prefecture[i]] = int(venue_i_t[i])


In [19]:
venue_key = sorted(set(train["venue"].values.tolist()),key =train["venue"].values.tolist().index)
l =[]
a = []
for i in range(len(venue_key)):
    ave = sum(train["attendance"][train["venue"] == venue_key[i]].values)/len(train["attendance"][train["venue"] == venue_key[i]].values)
    l.append(ave)
    a.append(ave)
a.sort()
venue_t =[0]*len(l)
for i in range(len(l)):
    venue_t[i] = a.index(l[i])

for i in range(len(l)):
    train["venue"][train["venue"] == venue_key[i]] = int(venue_t[i])

<ipython-input-19-f388e3102567>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["venue"][train["venue"] == venue_key[i]] = int(venue_t[i])


放送局

In [20]:
"""""
broadcasters_key = sorted(set(train["broadcasters"].values.tolist()),key =train["broadcasters"].values.tolist().index)
sum(train["attendance"][train["broadcasters"] == broadcasters_key[0]].values)/len(train["attendance"][train["broadcasters"] == broadcasters_key[0]].values)
l =[]
a = []
for i in range(len(broadcasters_key)):
    ave = sum(train["attendance"][train["broadcasters"] == broadcasters_key[i]].values)/len(train["attendance"][train["broadcasters"] == broadcasters_key[i]].values)
    l.append(ave)
    a.append(ave)
a.sort()
broad_t =[0]*len(l)
for i in range(len(l)):
    broad_t[i] = a.index(l[i])

for i in range(len(l)):
    train["broadcasters"][train["broadcasters"] == broadcasters_key[i]] = int(broad_t[i])

train.head()
"""

'""\nbroadcasters_key = sorted(set(train["broadcasters"].values.tolist()),key =train["broadcasters"].values.tolist().index)\nsum(train["attendance"][train["broadcasters"] == broadcasters_key[0]].values)/len(train["attendance"][train["broadcasters"] == broadcasters_key[0]].values)\nl =[]\na = []\nfor i in range(len(broadcasters_key)):\n    ave = sum(train["attendance"][train["broadcasters"] == broadcasters_key[i]].values)/len(train["attendance"][train["broadcasters"] == broadcasters_key[i]].values)\n    l.append(ave)\n    a.append(ave)\na.sort()\nbroad_t =[0]*len(l)\nfor i in range(len(l)):\n    broad_t[i] = a.index(l[i])\n\nfor i in range(len(l)):\n    train["broadcasters"][train["broadcasters"] == broadcasters_key[i]] = int(broad_t[i])\n    \ntrain.head()\n'

テストデータ前処理

In [21]:
test["match_date"] = [int(x.replace("-","")) for x in test["match_date"].values]
test["kick_off_time"] = [int(x.replace(":","")) for x in test["kick_off_time"].values]
test["section"] = [int(x.replace("第","").replace("節","")) for x in test["section"].values]
test["round"] = [int(x.replace("第","").replace("日","")) for x in test["round"].values]



In [22]:
for i in range(len(home_team_key)):
    test["home_team"][test["home_team"] == home_team_key[i]] = int(home_t[i])

test["home_team"][test["home_team"] == "長崎"] = -1

for i in range(len(home_team_key)):
    test["away_team"][test["away_team"] == home_team_key[i]] = int(home_t[i])

test["away_team"][test["away_team"] == "長崎"] = -1

for i in range(len(weather_key)):
    test["weather"][test["weather"] == weather_key[i]] = int(weather_t[i])
test["weather"][test["weather"] == "曇晴"] = -1
test["weather"][test["weather"] == "曇のち霧"] = -2
test["weather"][test["weather"] == "雨のち曇のち雨"] = -3
test["weather"][test["weather"] == "曇時々雨のち曇"] = -4



for i in range(len(venue_key)):
    test["venue"][test["venue"] == venue_key[i]] = int(venue_t[i])

test["venue"][test["venue"] == "パナソニックスタジアム吹田"] = -1
test["venue"][test["venue"] == "三協フロンテア柏スタジアム"] = -2
test["venue"][test["venue"] == "トランスコスモススタジアム長崎"] = -3

for i in range(len(prefecture)):
    test["address"][test["address"] == prefecture[i]] = int(venue_i_t[i])

"""""
for i in range(len(broadcasters_key)):
    test["broadcasters"][test["broadcasters"] == broadcasters_key[i]] = int(broad_t[i])

test["broadcasters"][test["broadcasters"] == "DAZN/テレビ神奈川/テレ玉"] = -1
test["broadcasters"][test["broadcasters"] == "DAZN/NHK仙台/NHK札幌"] = -2
test["broadcasters"][test["broadcasters"] == "DAZN/NHK総合"] = -3

"""""


<ipython-input-22-3c4a40caddb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["home_team"][test["home_team"] == home_team_key[i]] = int(home_t[i])
<ipython-input-22-3c4a40caddb4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["home_team"][test["home_team"] == "長崎"] = -1
<ipython-input-22-3c4a40caddb4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["away_team"][test["away_team"] == home_team_key[i]] = int(home_t[i]

'""   \nfor i in range(len(broadcasters_key)):\n    test["broadcasters"][test["broadcasters"] == broadcasters_key[i]] = int(broad_t[i])\n\ntest["broadcasters"][test["broadcasters"] == "DAZN/テレビ神奈川/テレ玉"] = -1\ntest["broadcasters"][test["broadcasters"] == "DAZN/NHK仙台/NHK札幌"] = -2\ntest["broadcasters"][test["broadcasters"] == "DAZN/NHK総合"] = -3\n\n'

休日データ統合

In [23]:
holiday = pd.read_csv("/content/drive/MyDrive/SIGNATE/Deloitte/holidays_in_japan.csv")

In [24]:
holiday["holiday_date"] = [int(x.replace("-","")) for x in holiday["holiday_date"].values]

In [25]:
holiday_key = sorted(set(holiday["holiday_date"].values.tolist()),key =holiday["holiday_date"].values.tolist().index)

In [26]:
train["holiday"]=0
test["holiday"] =0
for i in range(len(holiday_key)):
    train["holiday"][train["match_date"]==holiday_key[i]] = 1
    test["holiday"][test["match_date"]==holiday_key[i]] = 1

<ipython-input-26-83581872df2e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["holiday"][train["match_date"]==holiday_key[i]] = 1
<ipython-input-26-83581872df2e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["holiday"][test["match_date"]==holiday_key[i]] = 1


試合詳細データ追加

In [27]:
match_r = pd.read_csv("/content/drive/MyDrive/SIGNATE/Deloitte/match_reports.csv")
match_r.columns
match_c =['home_team_score','away_team_score']

In [28]:
train["home_team_score"]= 0
train["away_team_score"]= 0
test["home_team_score"]= 0
test["away_team_score"]= 0
match_r_key = sorted(set(match_r["id"].values.tolist()),key =match_r["id"].values.tolist().index)
for i in range(len(match_r_key)):
    train["home_team_score"][train["id"]==match_r_key[i]] = match_r["home_team_score"][i]
    train["away_team_score"][train["id"]==match_r_key[i]] = match_r["away_team_score"][i]
    test["home_team_score"][test["id"]==match_r_key[i]] = match_r["home_team_score"][i]
    test["away_team_score"][test["id"]==match_r_key[i]] = match_r["away_team_score"][i]

test.head()

<ipython-input-28-eb63aa7a8095>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["home_team_score"][train["id"]==match_r_key[i]] = match_r["home_team_score"][i]
<ipython-input-28-eb63aa7a8095>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["away_team_score"][train["id"]==match_r_key[i]] = match_r["away_team_score"][i]
<ipython-input-28-eb63aa7a8095>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["home_team_score"][

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,broadcasters,capacity,address,holiday,home_team_score,away_team_score
0,19075,20170225,1235,1,1,25,28,55,29,10.2,45.0,DAZN/テレビ神奈川/テレ玉,72081,42,0,3,2
1,19076,20170225,1404,1,1,17,9,36,26,10.6,36.0,DAZN/NHK仙台/NHK札幌,19694,29,0,1,0
2,19077,20170225,1404,1,1,22,26,43,29,10.1,63.0,DAZN/NHK総合,37496,37,0,0,1
3,19078,20170225,1404,1,1,16,13,32,29,15.0,40.0,DAZN/NHK BS1/テレビ静岡,20248,26,0,0,1
4,19079,20170225,1404,1,1,14,27,37,29,11.2,45.0,DAZN/NHK広島/NHK新潟,36894,24,0,1,1


スコア差

In [29]:
train["score_d"]=train["home_team_score"]-train["away_team_score"]
test["score_d"]=test["home_team_score"]-test["away_team_score"]


勝敗

In [30]:
train["win"] = 0
test["win"] = 0
train["win"][train["score_d"]>0] = 1
train["win"][train["score_d"]<0] = -1
test["win"][test["score_d"]>0] = 1
test["win"][test["score_d"]<0] = -1

<ipython-input-30-4d4238999156>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["win"][train["score_d"]>0] = 1
<ipython-input-30-4d4238999156>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["win"][train["score_d"]<0] = -1
<ipython-input-30-4d4238999156>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["win"][test["score_d"]>0] = 1
<ipython-input-30-4d4238999156>:6: SettingWithCopyWarning: 
A value is trying to be se

勝ち負け率

In [31]:
train["win_rate"] = 0
test["win_rate"] = 0
for i in range(len(home_t)):
    h_w= sum(train["win"][train["home_team"]==home_t[i]])-sum(train["win"][train["away_team"]==home_t[i]])
    n = sum(train["home_team"][train["home_team"]==home_t[i]])-sum(train["home_team"][train["away_team"]==home_t[i]])
    train["win_rate"][train["home_team"]==home_t[i]] =h_w/n
    h_w= sum(test["win"][test["home_team"]==home_t[i]])-sum(test["win"][test["away_team"]==home_t[i]])
    n = sum(test["home_team"][test["home_team"]==home_t[i]])-sum(test["home_team"][test["away_team"]==home_t[i]])
    if n != 0:
        test["win_rate"][test["home_team"]==home_t[i]] =h_w/n
    else:
        test["win_rate"][test["home_team"]==home_t[i]] =0

<ipython-input-31-225a79351d54>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["win_rate"][train["home_team"]==home_t[i]] =h_w/n
<ipython-input-31-225a79351d54>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["win_rate"][test["home_team"]==home_t[i]] =h_w/n


In [32]:
train.head()

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,...,broadcasters,attendance,capacity,address,holiday,home_team_score,away_team_score,score_d,win,win_rate
0,9190,20060304,1604,1,1,21,28,40,29,8.3,...,NHK総合/J SPORTS(録),20916,21000,36,0,1,1,0,0,0.098476
1,9191,20060305,1300,1,2,6,16,23,29,12.9,...,山梨放送/テレビ静岡(録)/J SPORTS(録),14277,15859,9,0,0,2,-2,-1,0.040657
2,9192,20060305,1335,1,2,26,24,51,29,12.1,...,BS-i/MXテレビ(録)/J SPORTS(録),22531,48999,43,0,2,0,2,1,0.008074
3,9193,20060305,1404,1,2,10,7,56,29,11.6,...,J SPORTS,28564,51697,26,0,1,1,0,0,0.026575
4,9194,20060305,1404,1,2,19,18,25,29,13.1,...,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),17199,20223,31,0,3,2,1,1,0.059041


型確認

In [33]:
train["home_team"]=train["home_team"].astype("int64")
train["away_team"]=train["away_team"].astype("int64")
train["venue"]=train["venue"].astype("int64")
train["weather"]=train["weather"].astype("int64")
train["address"]=train["address"].astype("int64")
train["temperature"]=train["temperature"].astype("float64")
#train["broadcasters"]=train["broadcasters"].astype("int64")

In [34]:
test["home_team"]=test["home_team"].astype("int64")
test["away_team"]=test["away_team"].astype("int64")
test["venue"]=test["venue"].astype("int64")
test["weather"]=test["weather"].astype("int64")
test["address"]=test["address"].astype("int64")
train["temperature"]=train["temperature"].astype("float64")
train["humidity"]=train["humidity"].astype("float64")

#test["broadcasters"]=test["broadcasters"].astype("int64")

In [35]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3366 entries, 0 to 3365
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3366 non-null   int64  
 1   match_date       3366 non-null   int64  
 2   kick_off_time    3366 non-null   int64  
 3   section          3366 non-null   int64  
 4   round            3366 non-null   int64  
 5   home_team        3366 non-null   int64  
 6   away_team        3366 non-null   int64  
 7   venue            3366 non-null   int64  
 8   weather          3366 non-null   int64  
 9   temperature      3366 non-null   float64
 10  humidity         3366 non-null   float64
 11  broadcasters     3366 non-null   object 
 12  attendance       3366 non-null   int64  
 13  capacity         3366 non-null   int64  
 14  address          3366 non-null   int64  
 15  holiday          3366 non-null   int64  
 16  home_team_score  3366 non-null   int64  
 17  away_team_scor

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               459 non-null    int64  
 1   match_date       459 non-null    int64  
 2   kick_off_time    459 non-null    int64  
 3   section          459 non-null    int64  
 4   round            459 non-null    int64  
 5   home_team        459 non-null    int64  
 6   away_team        459 non-null    int64  
 7   venue            459 non-null    int64  
 8   weather          459 non-null    int64  
 9   temperature      459 non-null    float64
 10  humidity         459 non-null    float64
 11  broadcasters     459 non-null    object 
 12  capacity         459 non-null    int64  
 13  address          459 non-null    int64  
 14  holiday          459 non-null    int64  
 15  home_team_score  459 non-null    int64  
 16  away_team_score  459 non-null    int64  
 17  score_d         

None

正規化

In [36]:
#train["temperature"] = (train["temperature"] - train["temperature"].mean()) / train["temperature"].std(ddof=0)
#test["temperature"] = (test["temperature"] - test["temperature"].mean()) / test["temperature"].std(ddof=0)

In [37]:
#train["humidity"] = (train["humidity"] - train["humidity"].mean()) / train["humidity"].std(ddof=0)
#test["humidity"] = (test["humidity"] - test["humidity"].mean()) / test["humidity"].std(ddof=0)

In [38]:
train["kick_off_time"] =((train["kick_off_time"] - 1700)%100)/60+(train["kick_off_time"] - 1700)//100
train["kick_off_time"] = (train["kick_off_time"] - train["kick_off_time"].mean()) / train["kick_off_time"].std(ddof=0)
test["kick_off_time"] =((test["kick_off_time"] - 1700)%100)/60+(test["kick_off_time"] - 1700)//100
test["kick_off_time"] = (test["kick_off_time"] - test["kick_off_time"].mean()) / test["kick_off_time"].std(ddof=0)

In [39]:
#train_std=train["attendance"].std(ddof=0)
#train_mean=train["attendance"].mean()
#train["attendance"] = (train["attendance"] - train["attendance"].mean()) / train["attendance"].std(ddof=0)

In [40]:
#train["capacity"] = (train["capacity"] - train["capacity"].mean()) / train["capacity"].std(ddof=0)
#test["capacity"] = (test["capacity"] - test["capacity"].mean()) / test["capacity"].std(ddof=0)

In [41]:
train.head()

,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,...,broadcasters,attendance,capacity,address,holiday,home_team_score,away_team_score,score_d,win,win_rate
0,9190,20060304,-0.270798,1,1,21,28,40,29,8.3,...,NHK総合/J SPORTS(録),20916,21000,36,0,1,1,0,0,0.098476
1,9191,20060305,-1.658459,1,2,6,16,23,29,12.9,...,山梨放送/テレビ静岡(録)/J SPORTS(録),14277,15859,9,0,0,2,-2,-1,0.040657
2,9192,20060305,-1.394502,1,2,26,24,51,29,12.1,...,BS-i/MXテレビ(録)/J SPORTS(録),22531,48999,43,0,2,0,2,1,0.008074
3,9193,20060305,-1.175794,1,2,10,7,56,29,11.6,...,J SPORTS,28564,51697,26,0,1,1,0,0,0.026575
4,9194,20060305,-1.175794,1,2,19,18,25,29,13.1,...,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),17199,20223,31,0,3,2,1,1,0.059041


ターゲット最後に

In [42]:
first_column = train.pop('attendance')
l = len(train.columns.values)
train.insert(l,"attendance",first_column)

相関行列

In [43]:
train.corr().style.background_gradient(axis=None,low=(1 + train.corr().values.min()) / (1 - train.corr().values.min()))

<ipython-input-43-1300079c7306>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr().style.background_gradient(axis=None,low=(1 + train.corr().values.min()) / (1 - train.corr().values.min()))
<ipython-input-43-1300079c7306>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr().style.background_gradient(axis=None,low=(1 + train.corr().values.min()) / (1 - train.corr().values.min()))


,id,match_date,kick_off_time,section,round,home_team,away_team,venue,weather,temperature,humidity,capacity,address,holiday,home_team_score,away_team_score,score_d,win,win_rate,attendance
id,1.000000,0.998102,0.084049,-0.210118,-0.081576,-0.038356,-0.038336,-0.044761,0.032639,0.091491,0.028141,-0.032698,-0.027569,-0.012514,-0.052926,-0.010363,-0.032159,-0.039737,-0.070071,-0.047427
match_date,0.998102,1.000000,0.082726,-0.219694,-0.083627,-0.037694,-0.037709,-0.045251,0.029588,0.081233,0.022911,-0.032779,-0.026945,-0.010497,-0.052848,-0.011266,-0.031501,-0.038267,-0.074332,-0.051103
kick_off_time,0.084049,0.082726,1.000000,0.009390,-0.017509,-0.013175,0.032303,-0.041948,-0.019661,0.394330,0.501724,-0.050230,0.007141,-0.142882,0.049123,0.005430,0.032634,0.028845,0.030342,-0.083261
section,-0.210118,-0.219694,0.009390,1.000000,-0.086946,0.010504,0.005904,-0.014531,-0.000479,0.105779,0.135653,-0.008421,0.003893,-0.029482,0.023564,-0.002439,0.019010,0.009410,0.006686,0.048659
round,-0.081576,-0.083627,-0.017509,-0.086946,1.000000,-0.002546,0.013464,-0.010871,0.009096,0.036698,-0.012252,-0.021227,0.006161,-0.113046,0.016394,-0.016015,0.022751,0.027588,0.031347,0.003333
home_team,-0.038356,-0.037694,-0.013175,0.010504,-0.002546,1.000000,-0.054209,0.832093,0.041341,-0.027386,-0.027141,0.629087,0.668012,0.020305,0.129493,-0.094473,0.158401,0.135401,0.001371,0.613076
away_team,-0.038336,-0.037709,0.032303,0.005904,0.013464,-0.054209,1.000000,0.013613,0.014645,0.002366,0.007317,0.028758,-0.019615,-0.030221,-0.103881,0.099372,-0.142761,-0.121504,0.001371,0.143746
venue,-0.044761,-0.045251,-0.041948,-0.014531,-0.010871,0.832093,0.013613,1.000000,0.058462,-0.056854,-0.046873,0.817125,0.589849,0.016801,0.081959,-0.064682,0.103508,0.085392,0.004094,0.719710
weather,0.032639,0.029588,-0.019661,-0.000479,0.009096,0.041341,0.014645,0.058462,1.000000,0.150580,-0.354569,0.065684,-0.002398,0.039461,0.014982,0.008758,0.005228,-0.010624,0.001315,0.094451
temperature,0.091491,0.081233,0.394330,0.105779,0.036698,-0.027386,0.002366,-0.056854,0.150580,1.000000,0.244572,-0.034531,-0.016109,-0.057414,0.036637,0.018506,0.014722,0.007686,0.026090,-0.041814


学習データセット用意

In [44]:
%%time
from tqdm import tqdm
import scipy.stats
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

CPU times: user 473 ms, sys: 160 ms, total: 633 ms
Wall time: 536 ms


In [86]:
c = train.columns.values

features = ['match_date', 'kick_off_time', 'section', 'round',
       'home_team', 'away_team', 'venue', 'weather', 'temperature',
       'humidity', 'holiday',"capacity","address","win_rate","home_team_score","away_team_score","score_d","win"]

#features = ['home_team', 'venue', 'weather', 'humidity', 'holiday',"capacity","address"]

y = train["attendance"]
X = train[features]
"""train_X, val_X, train_y, val_y = train_test_split(X, y,                # 訓練データとテストデータに分割する
                                                    test_size=0.3,       # テストデータの割合
                                                    shuffle=True,        # シャッフルする
                                                    random_state=0)
 """
train_y = train["attendance"]
train_X = train[features]

In [46]:
from sklearn.linear_model    import LinearRegression
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics         import mean_squared_error

重回帰

In [47]:
"""
lm = LinearRegression()
lm.fit(train_X, train_y)
y_pred2 = lm.predict(vali_X)
lm_mse = mean_squared_error(vali_y, y_pred2)
print('LinerRegression RMSE: ', np.sqrt(lm_mse))

coef_pd = pd.DataFrame(lm.coef_, columns=['coef'], index=features)

coef_pd.sort_values('coef', ascending=True)
"""

"\nlm = LinearRegression()\nlm.fit(train_X, train_y)\ny_pred2 = lm.predict(vali_X)\nlm_mse = mean_squared_error(vali_y, y_pred2)\nprint('LinerRegression RMSE: ', np.sqrt(lm_mse))\n\ncoef_pd = pd.DataFrame(lm.coef_, columns=['coef'], index=features)\n\ncoef_pd.sort_values('coef', ascending=True)\n"

In [87]:
test_X = test[features]
#test_y = lm.predict(test_X)

データファイル書き出し

In [ ]:
import datetime
dt_now = datetime.datetime.now()
name=dt_now.strftime('%Y%m%d_%H%M')
file_name ="{}.csv".format(name)

# PassengerIdを取得
index = np.array(test["id"]).astype(int)

# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame({'id': index, 'target': test_y})

# 書き出し
my_solution.to_csv(file_name,header=False,index=False)

NameError: name 'test_y' is not defined

In [85]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import lightgbm as lgb

In [88]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_X)
    rmse= np.sqrt(-cross_val_score(model, train_X, train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [90]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(silent=False,n_jobs=-1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [60]:
cv_params ={'max_depth':[3,4,5,6,7,8],
            'min_child_weight':[1,2,3,4,5],
            'gamma':[i/10.0 for i in range(0,6)],
            'subsample':[i/10.0 for i in range(6,11)],
            'colsample_bytree':[i/10.0 for i in range(6,11)],
            'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
            'n_estimators':[1000,2000],
            'reg_lambda':[1e-5, 1e-2, 0.1, 1],
            'learning_rate':[0.1,0.2,0.3]
            }
model_xgb = RandomizedSearchCV(model, cv_params, n_iter=200, cv=5, n_jobs=-1)

In [63]:
model_xgb.fit(train_X,train_y,
early_stopping_rounds=50,
eval_set=[(val_X, val_y)],
eval_metric='rmse',
verbose=0)
print('optimal_parameters\n', model_xgb.best_params_)
"""output
optimal_parameters
 {'subsample': 0.8, 'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 1.0}
"""

#パラメータが最適化されたモデルを作成
opt_model = model_xgb.best_estimator_
y_pred_train = opt_model.predict(train_X)
y_pred_test = opt_model.predict(val_X)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[03:10:06] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

optimal_parameters
 {'subsample': 0.9, 'reg_lambda': 0.1, 'reg_alpha': 0.01, 'n_estimators': 2000, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8}


In [91]:
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Lasso score: 6113.4659 (1066.2536)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.039e+09, tolerance: 2.500e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.042e+09, tolerance: 2.509e+07
  model = cd_fast.enet_coordinate_descent(


In [92]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.996e+10, tolerance: 2.152e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.473e+10, tolerance: 2.321e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.153e+10, tolerance: 2.578e

ElasticNet score: 6113.4397 (1066.2774)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.872e+10, tolerance: 2.509e+07
  model = cd_fast.enet_coordinate_descent(


In [93]:
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Kernel Ridge score: 6129.7341 (989.7184)


In [94]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Gradient Boosting score: 4816.4932 (615.9591)


In [95]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

[03:59:44] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[03:59:47] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[03:59:47] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[03:59:47] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[03:59:48] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

Xgboost score: 4801.9726 (320.8017)


In [96]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})" .format(score.mean(), score.std()))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6

In [97]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self

    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [98]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.996e+10, tolerance: 2.152e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.473e+10, tolerance: 2.321e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving d

 Averaged base models score: 5433.1705 (803.2601)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.042e+09, tolerance: 2.509e+07
  model = cd_fast.enet_coordinate_descent(


In [99]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [100]:
averaged_models.fit(train_X, train_y)
averaged_train_pred = averaged_models.predict(train_X)
test_y_ave=averaged_models.predict(test_X)
averaged_pred = np.expm1(averaged_models.predict(test_X))
print(rmsle(train_y, averaged_train_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.897e+10, tolerance: 3.017e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.122e+10, tolerance: 3.017e+07
  model = cd_fast.enet_coordinate_descent(


4822.528157573773


<ipython-input-100-59cd633b0b8a>:4: RuntimeWarning: overflow encountered in expm1
  averaged_pred = np.expm1(averaged_models.predict(test_X))


In [101]:
model_xgb.fit(train_X, train_y)
xgb_train_pred = model_xgb.predict(train_X)
test_y_xgb= opt_model.predict(test_X)
xgb_pred = np.expm1(model_xgb.predict(test_X))
print(rmsle(train_y, xgb_train_pred))

[04:04:28] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

1095.0107509351094


<ipython-input-101-a6fb68f9894b>:4: RuntimeWarning: overflow encountered in expm1
  xgb_pred = np.expm1(model_xgb.predict(test_X))


In [111]:
model_lgb.fit(train_X, train_y)
lgb_train_pred = model_lgb.predict(train_X)
test_y_lgb= opt_model.predict(test_X)
lgb_pred = np.expm1(model_lgb.predict(test_X))
print(rmsle(train_y, lgb_train_pred))

[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
3726.4379662585625


<ipython-input-111-c157109951f7>:4: RuntimeWarning: overflow encountered in expm1
  lgb_pred = np.expm1(model_lgb.predict(test_X))


In [104]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(train_y,averaged_train_pred*0.70 +
               xgb_train_pred*0.15 + lgb_train_pred*0.15 ))

RMSLE score on train data:
3916.004049760809


In [112]:
ensemble = test_y_ave*0.70 +test_y_xgb*0.15 + test_y_lgb*0.15

In [116]:
test_y_xgb

array([50840.848 , 11066.526 , 22407.805 ,  9017.182 , 18358.797 ,
       14723.781 , 25011.785 , 14205.408 , 30831.176 , 42739.723 ,
       13580.15  , 17046.074 , 26014.625 ,  9969.049 , 18943.65  ,
       18410.916 , 12942.383 , 21707.217 , 26629.215 , 21862.357 ,
       21724.219 , 17041.297 ,  8519.352 , 11471.1455, 17622.252 ,
        7967.988 , 28382.523 , 15593.067 , 25130.14  , 10710.234 ,
       18625.832 ,  6290.9443,  9714.704 , 18783.996 , 27033.514 ,
       36194.492 , 11577.787 , 23844.21  , 15349.477 , 23535.793 ,
       15617.757 , 15496.117 , 23866.498 , 12504.975 , 12159.656 ,
       23278.64  , 26838.273 , 11923.089 , 14536.078 ,  5108.2935,
        4844.3423,  8971.412 , 18890.121 , 23418.025 , 21313.8   ,
       13019.279 ,  7446.9526, 32416.71  , 32004.89  , 11737.349 ,
       17248.836 , 17580.652 , 21999.904 , 24092.107 , 25758.824 ,
       33184.06  , 11322.804 , 15179.1045, 16031.611 ,  7823.8174,
       24589.785 , 15740.487 , 12476.2705, 22387.367 , 18099.2

In [ ]:
import datetime
dt_now = datetime.datetime.now()
name=dt_now.strftime('%Y%m%d_%H%M')
file_name ="{}.csv".format(name)

# PassengerIdを取得
index = np.array(test["id"]).astype(int)

# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame({'id': index, 'target': test_y_ave})

# 書き出し
my_solution.to_csv(file_name,header=False,index=False)

ニューラルネットワーク

In [83]:
import numpy as np
import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout


# ニューラルネットワークのモデルを定義
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(train_X.shape[1], )))
model.add(Dense(64,activation='relu',input_shape=(train_X.shape[1], )))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu',input_shape=(train_X.shape[1], )))
model.add(Dense(1))

# モデルのコンパイル
model.compile(loss='mean_squared_error', optimizer='adam')

# モデルの学習
model.fit(train_X, train_y, epochs=300, batch_size=64, validation_data=(val_X, val_y))

# モデルの予測
y_pred = model.predict(test_X)


Epoch 1/300
37/37 [==============================] - 2s 8ms/step - loss: 530824691712.0000 - val_loss: 7618827264.0000
Epoch 2/300
37/37 [==============================] - 0s 5ms/step - loss: 63251705856.0000 - val_loss: 6488631808.0000
Epoch 3/300
37/37 [==============================] - 0s 4ms/step - loss: 28865224704.0000 - val_loss: 5363593216.0000
Epoch 4/300
37/37 [==============================] - 0s 4ms/step - loss: 14554690560.0000 - val_loss: 1648208512.0000
Epoch 5/300
37/37 [==============================] - 0s 5ms/step - loss: 8318498304.0000 - val_loss: 1870996864.0000
Epoch 6/300
37/37 [==============================] - 0s 5ms/step - loss: 6772899840.0000 - val_loss: 773592832.0000
Epoch 7/300
37/37 [==============================] - 0s 5ms/step - loss: 4912933888.0000 - val_loss: 368447904.0000
Epoch 8/300
37/37 [==============================] - 0s 5ms/step - loss: 3765373184.0000 - val_loss: 182669904.0000
Epoch 9/300
37/37 [==============================] - 0s 5ms/st

In [75]:
train_X.shape[0]

2356

In [84]:
y_pred=np.ravel(y_pred)
y_pred

array([26458.209 , 11124.766 , 16234.67  , 11231.029 , 16044.457 ,
       12152.311 , 19381.842 , 10311.322 , 16904.111 , 23694.57  ,
       10391.879 , 16043.081 , 19734.135 , 10221.146 , 13469.955 ,
       10304.37  , 10195.122 , 12992.468 , 16235.94  , 12993.551 ,
       23692.377 , 16940.082 , 10310.033 , 12155.418 , 17512.39  ,
       11124.601 , 16907.387 , 16940.422 , 26458.674 , 10390.771 ,
       11232.189 , 10196.93  , 10775.386 , 13467.035 , 19736.275 ,
       16908.861 , 11124.558 , 17513.299 , 16041.269 , 20545.2   ,
       10778.023 , 19732.562 , 13469.125 , 10309.4   , 10390.604 ,
       16902.182 , 23693.213 , 12990.789 , 16232.076 , 10306.826 ,
       10192.774 , 12154.636 , 16941.303 , 26452.314 , 16941.148 ,
       10393.333 , 11230.4   , 19738.98  , 19383.75  , 10219.357 ,
       13464.383 , 16041.425 , 11125.173 , 12993.321 , 16904.26  ,
       23692.764 , 10392.262 , 16044.351 , 16231.988 , 10197.762 ,
       17513.574 , 12153.561 , 11231.437 , 17511.861 , 10314.1

In [51]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime

In [53]:
import datetime
dt_now = datetime.datetime.now()
name=dt_now.strftime('%Y%m%d_%H%M')
file_name ="{}.csv".format(name)

# PassengerIdを取得
index = np.array(test["id"]).astype(int)

# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame({'id': index, 'target': y_pred})

# 書き出し
my_solution.to_csv("/content/drive/MyDrive/SIGNATE/Deloitte/{}".format(file_name),header=False,index=False)

In [52]:
import datetime
dt_now = datetime.datetime.now()
name=dt_now.strftime('%Y%m%d_%H%M')
file_name ="{}.csv".format(name)
file_name

'20230616_0252.csv'

In [63]:
%logstart -t

Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : False
Raw input log  : False
Timestamping   : True
State          : active
